# AlgoHouse Data Quality Benchmark

Comprehensive data quality analysis for crypto exchanges - detect wash trading, validate order books, and score data reliability.

**Runtime:** < 10 minutes for any quant to clone and execute

---

## Section 1: Setup

Install required dependencies and configure exchange list.

In [ ]:
# Install dependencies (run once)
!pip install -q ccxt pandas scipy statsmodels plotly numpy requests

import ccxt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chisquare
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import requests
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Exchanges to test (10 major exchanges)
EXCHANGES_TO_TEST = [
    'binance', 'kraken', 'coinbase', 'bybit', 'kucoin',
    'gate', 'okx', 'huobi', 'mexc', 'bitget'
]

SYMBOL = 'BTC/USDT'
SAMPLE_SIZE = 1000  # trades per exchange

print(f"✅ Setup complete! Testing {len(EXCHANGES_TO_TEST)} exchanges for {SYMBOL}")
print(f"📊 Data points per exchange: {SAMPLE_SIZE} trades + order book + 24h OHLCV")

---

## Section 2: Data Collection

Fetch data from:
1. **CCXT**: 1000 recent trades, L2 order book (20 levels), 24h OHLCV
2. **AlgoHouse API**: Exchange quality scores
3. **Coin Metrics Community API**: Reference data

In [ ]:
def fetch_exchange_data(exchange_id):
    """Fetch all required data for a single exchange."""
    try:
        # Initialize exchange
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class({
            'enableRateLimit': True,
            'timeout': 30000
        })
        
        data = {
            'exchange': exchange_id,
            'timestamp': datetime.now().isoformat(),
            'symbol': SYMBOL
        }
        
        # 1. Fetch recent trades
        print(f"  Fetching trades for {exchange_id}...", end=' ')
        trades = exchange.fetch_trades(SYMBOL, limit=SAMPLE_SIZE)
        data['trades'] = trades
        data['trade_count'] = len(trades)
        print(f"✓ {len(trades)} trades")
        
        # 2. Fetch L2 order book (20 levels)
        print(f"  Fetching order book for {exchange_id}...", end=' ')
        orderbook = exchange.fetch_order_book(SYMBOL, limit=20)
        data['orderbook'] = orderbook
        data['bid_levels'] = len(orderbook['bids'])
        data['ask_levels'] = len(orderbook['asks'])
        print(f"✓ {len(orderbook['bids'])} bids, {len(orderbook['asks'])} asks")
        
        # 3. Fetch 24h OHLCV
        print(f"  Fetching 24h OHLCV for {exchange_id}...", end=' ')
        since = exchange.milliseconds() - 24 * 60 * 60 * 1000
        ohlcv = exchange.fetch_ohlcv(SYMBOL, '1h', since=since, limit=24)
        data['ohlcv'] = ohlcv
        data['ohlcv_count'] = len(ohlcv)
        print(f"✓ {len(ohlcv)} candles")
        
        # 4. Calculate 24h volume
        ticker = exchange.fetch_ticker(SYMBOL)
        data['volume_24h'] = ticker.get('quoteVolume', 0)
        
        return data
        
    except Exception as e:
        print(f"  ✗ Error: {str(e)[:100]}")
        return {
            'exchange': exchange_id,
            'error': str(e),
            'trades': [],
            'trade_count': 0
        }

# Fetch data from all exchanges
print("\n🔄 Fetching data from all exchanges...\n")
exchange_data = {}
for exchange_id in EXCHANGES_TO_TEST:
    print(f"📡 {exchange_id.upper()}")
    exchange_data[exchange_id] = fetch_exchange_data(exchange_id)
    time.sleep(1)  # Rate limiting

print(f"\n✅ Data collection complete! {len([e for e in exchange_data.values() if e.get('trade_count', 0) > 0])}/{len(EXCHANGES_TO_TEST)} exchanges successful")

In [ ]:
# Fetch AlgoHouse quality scores
def fetch_algohouse_scores():
    """Fetch exchange quality scores from AlgoHouse API."""
    try:
        response = requests.get('https://api.algohouse.com/v1/exchanges', timeout=10)
        if response.status_code == 200:
            return {e['id']: e for e in response.json()}
        else:
            print(f"⚠️  AlgoHouse API returned {response.status_code}")
            return {}
    except Exception as e:
        print(f"⚠️  AlgoHouse API error: {str(e)}")
        return {}

print("\n📊 Fetching AlgoHouse quality scores...")
algohouse_scores = fetch_algohouse_scores()
print(f"✅ Retrieved scores for {len(algohouse_scores)} exchanges")

# Fetch Coin Metrics data (Community API - free tier)
def fetch_coinmetrics_data():
    """Fetch reference data from Coin Metrics Community API."""
    try:
        # Coin Metrics Community API endpoint for markets
        url = 'https://community-api.coinmetrics.io/v4/catalog-all/markets'
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            return data.get('data', [])
        else:
            print(f"⚠️  Coin Metrics API returned {response.status_code}")
            return []
    except Exception as e:
        print(f"⚠️  Coin Metrics API error: {str(e)}")
        return []

print("\n📊 Fetching Coin Metrics reference data...")
coinmetrics_data = fetch_coinmetrics_data()
print(f"✅ Retrieved {len(coinmetrics_data)} markets from Coin Metrics")

---

## Section 3: Five Quality Measurements

### 3.1 Tick Completeness
Measure: % of expected trades present (no gaps > 1 second)

In [ ]:
def measure_tick_completeness(trades):
    """Measure tick completeness - flag gaps > 1 second."""
    if len(trades) < 2:
        return {'score': 0, 'gaps': 0, 'max_gap_seconds': 0}
    
    # Calculate time gaps between consecutive trades
    timestamps = [t['timestamp'] for t in trades]
    gaps = np.diff(timestamps) / 1000  # Convert to seconds
    
    # Count gaps > 1 second
    large_gaps = np.sum(gaps > 1.0)
    max_gap = np.max(gaps)
    
    # Score: 100 if no large gaps, penalize 5 points per gap
    score = max(0, 100 - (large_gaps * 5))
    
    return {
        'score': score,
        'gaps_over_1s': int(large_gaps),
        'max_gap_seconds': float(max_gap),
        'avg_gap_ms': float(np.mean(gaps) * 1000)
    }

# Measure tick completeness for all exchanges
tick_results = {}
for exchange_id, data in exchange_data.items():
    if data.get('trades'):
        tick_results[exchange_id] = measure_tick_completeness(data['trades'])

print("\n📏 Tick Completeness Scores:\n")
for exchange, result in sorted(tick_results.items(), key=lambda x: x[1]['score'], reverse=True):
    print(f"{exchange:12} Score: {result['score']:3.0f}/100  |  Gaps>1s: {result['gaps_over_1s']:3}  |  Max gap: {result['max_gap_seconds']:6.2f}s")

### 3.2 Order Book Depth Accuracy
Measure: Bid-ask spread reasonableness + depth at 0.1% price levels

In [ ]:
def measure_orderbook_accuracy(orderbook, recent_price):
    """Measure order book depth accuracy."""
    if not orderbook.get('bids') or not orderbook.get('asks'):
        return {'score': 0, 'spread_bps': 0, 'depth_quality': 'POOR'}
    
    bids = orderbook['bids']
    asks = orderbook['asks']
    
    # Best bid/ask
    best_bid = bids[0][0] if len(bids) > 0 else 0
    best_ask = asks[0][0] if len(asks) > 0 else 0
    
    if best_bid == 0 or best_ask == 0:
        return {'score': 0, 'spread_bps': 0, 'depth_quality': 'POOR'}
    
    # Spread in basis points
    spread_bps = ((best_ask - best_bid) / best_bid) * 10000
    
    # Depth at 0.1% from mid (institutional standard)
    mid_price = (best_bid + best_ask) / 2
    target_bid = mid_price * 0.999  # -0.1%
    target_ask = mid_price * 1.001  # +0.1%
    
    bid_depth = sum(level[1] for level in bids if level[0] >= target_bid)
    ask_depth = sum(level[1] for level in asks if level[0] <= target_ask)
    total_depth = bid_depth + ask_depth
    
    # Score components
    spread_score = max(0, 100 - (spread_bps * 10))  # Penalize wide spreads
    depth_score = min(100, total_depth * 10)  # Reward deep books
    
    # Combined score
    score = (spread_score * 0.6) + (depth_score * 0.4)
    
    # Quality label
    if score >= 80:
        quality = 'EXCELLENT'
    elif score >= 60:
        quality = 'GOOD'
    elif score >= 40:
        quality = 'FAIR'
    else:
        quality = 'POOR'
    
    return {
        'score': score,
        'spread_bps': spread_bps,
        'bid_depth': bid_depth,
        'ask_depth': ask_depth,
        'depth_quality': quality
    }

# Measure order book accuracy
orderbook_results = {}
for exchange_id, data in exchange_data.items():
    if data.get('orderbook') and data.get('trades'):
        recent_price = data['trades'][-1]['price'] if data['trades'] else 0
        orderbook_results[exchange_id] = measure_orderbook_accuracy(data['orderbook'], recent_price)

print("\n📖 Order Book Depth Accuracy:\n")
for exchange, result in sorted(orderbook_results.items(), key=lambda x: x[1]['score'], reverse=True):
    print(f"{exchange:12} Score: {result['score']:6.2f}/100  |  Spread: {result['spread_bps']:6.2f} bps  |  Quality: {result['depth_quality']}")

### 3.3 Benford's Law Wash Trading Test

**Academic Source:**  
Benford's Law states that in naturally occurring datasets, the first digit follows a logarithmic distribution.  
Wash trading (artificial volume) violates this distribution.

**References:**
- Nigrini, M. (1999). "I've Got Your Number." *Journal of Accountancy*
- Cong et al. (2022). "Crypto Wash Trading." *Yale/NBER Working Paper*

In [ ]:
def benford_law_test(trades):
    """
    Test trade volumes against Benford's Law distribution.
    
    Benford's Law: P(first_digit = d) = log10(1 + 1/d)
    Expected distribution: [30.1%, 17.6%, 12.5%, 9.7%, 7.9%, 6.7%, 5.8%, 5.1%, 4.6%]
    
    Chi-squared test: H0 = trades follow Benford's Law (natural)
    If p-value < 0.05: REJECT H0 → likely wash trading
    """
    if len(trades) < 100:
        return {'result': 'INSUFFICIENT_DATA', 'p_value': 0, 'chi_squared': 0}
    
    # Extract first digit from trade amounts (volume * price)
    amounts = [t['amount'] * t['price'] for t in trades if t.get('amount') and t.get('price')]
    first_digits = [int(str(abs(a))[0]) for a in amounts if abs(a) >= 1]
    
    if len(first_digits) < 100:
        return {'result': 'INSUFFICIENT_DATA', 'p_value': 0, 'chi_squared': 0}
    
    # Observed distribution
    observed = np.bincount(first_digits, minlength=10)[1:]  # Exclude 0
    
    # Expected distribution (Benford's Law)
    expected = np.array([np.log10(1 + 1/d) for d in range(1, 10)]) * len(first_digits)
    
    # Chi-squared test
    chi_stat, p_value = chisquare(observed, expected)
    
    # Result interpretation
    if p_value < 0.01:  # Strong evidence of manipulation
        result = 'FAIL'
        manipulation = 'HIGH'
    elif p_value < 0.05:  # Moderate evidence
        result = 'SUSPICIOUS'
        manipulation = 'MEDIUM'
    else:  # Follows Benford's Law (natural)
        result = 'PASS'
        manipulation = 'LOW'
    
    return {
        'result': result,
        'manipulation_risk': manipulation,
        'chi_squared': float(chi_stat),
        'p_value': float(p_value),
        'observed_dist': observed.tolist(),
        'expected_dist': expected.tolist()
    }

# Run Benford's Law test
benford_results = {}
for exchange_id, data in exchange_data.items():
    if data.get('trades'):
        benford_results[exchange_id] = benford_law_test(data['trades'])

print("\n🔬 Benford's Law Wash Trading Test:\n")
for exchange, result in sorted(benford_results.items(), key=lambda x: x[1].get('p_value', 0), reverse=True):
    print(f"{exchange:12} Result: {result['result']:15}  |  p-value: {result.get('p_value', 0):6.4f}  |  Risk: {result.get('manipulation_risk', 'N/A')}")

### 3.4 Buy/Sell Symmetry
Measure: Flag exchanges with 49-51% buy/sell imbalance (natural is ~50/50)

In [ ]:
def measure_buy_sell_symmetry(trades):
    """Measure buy/sell ratio - natural markets are ~50/50."""
    if len(trades) < 100:
        return {'result': 'INSUFFICIENT_DATA', 'buy_pct': 0, 'sell_pct': 0}
    
    # Infer side from price movement (if not explicitly provided)
    buy_count = 0
    sell_count = 0
    
    for i, trade in enumerate(trades):
        # If side is explicitly provided
        if 'side' in trade:
            if trade['side'] == 'buy':
                buy_count += 1
            else:
                sell_count += 1
        # Infer from price movement
        elif i > 0:
            if trade['price'] > trades[i-1]['price']:
                buy_count += 1
            else:
                sell_count += 1
    
    total = buy_count + sell_count
    if total == 0:
        return {'result': 'NO_DATA', 'buy_pct': 0, 'sell_pct': 0}
    
    buy_pct = (buy_count / total) * 100
    sell_pct = (sell_count / total) * 100
    
    # Natural markets have 45-55% buy ratio
    if 45 <= buy_pct <= 55:
        result = 'PASS'
    elif 40 <= buy_pct <= 60:
        result = 'ACCEPTABLE'
    else:
        result = 'SUSPICIOUS'
    
    return {
        'result': result,
        'buy_pct': buy_pct,
        'sell_pct': sell_pct,
        'buy_count': buy_count,
        'sell_count': sell_count
    }

# Measure buy/sell symmetry
symmetry_results = {}
for exchange_id, data in exchange_data.items():
    if data.get('trades'):
        symmetry_results[exchange_id] = measure_buy_sell_symmetry(data['trades'])

print("\n⚖️  Buy/Sell Symmetry:\n")
for exchange, result in sorted(symmetry_results.items(), key=lambda x: abs(x[1].get('buy_pct', 50) - 50)):
    print(f"{exchange:12} Result: {result['result']:12}  |  Buy: {result.get('buy_pct', 0):5.2f}%  |  Sell: {result.get('sell_pct', 0):5.2f}%")

### 3.5 Normalization Consistency
Measure: Timestamp alignment (all trades within 100ms bins)

In [ ]:
def measure_normalization_consistency(trades):
    """Measure timestamp normalization quality - should align to 100ms bins."""
    if len(trades) < 100:
        return {'score': 0, 'consistency': 'POOR'}
    
    # Check timestamp precision
    timestamps = [t['timestamp'] for t in trades]
    
    # Count how many timestamps align to 100ms boundaries
    aligned_count = sum(1 for ts in timestamps if ts % 100 == 0)
    alignment_pct = (aligned_count / len(timestamps)) * 100
    
    # Check for duplicate timestamps (indicates poor precision)
    unique_ts = len(set(timestamps))
    uniqueness_pct = (unique_ts / len(timestamps)) * 100
    
    # Score: penalize low uniqueness and poor alignment
    score = (uniqueness_pct * 0.7) + (min(alignment_pct, 50) * 0.6)  # Don't over-reward alignment
    
    if score >= 80:
        consistency = 'EXCELLENT'
    elif score >= 60:
        consistency = 'GOOD'
    elif score >= 40:
        consistency = 'FAIR'
    else:
        consistency = 'POOR'
    
    return {
        'score': score,
        'consistency': consistency,
        'uniqueness_pct': uniqueness_pct,
        'alignment_pct': alignment_pct
    }

# Measure normalization consistency
normalization_results = {}
for exchange_id, data in exchange_data.items():
    if data.get('trades'):
        normalization_results[exchange_id] = measure_normalization_consistency(data['trades'])

print("\n🕐 Timestamp Normalization Consistency:\n")
for exchange, result in sorted(normalization_results.items(), key=lambda x: x[1]['score'], reverse=True):
    print(f"{exchange:12} Score: {result['score']:6.2f}/100  |  Uniqueness: {result['uniqueness_pct']:5.2f}%  |  Quality: {result['consistency']}")

---

## Section 4: Composite Data Trust Score

Calculate overall data quality score (0-100) for each exchange.

In [ ]:
def calculate_trust_score(exchange_id):
    """
    Calculate composite Data Trust Score (0-100) from all measurements.
    
    Weights:
    - Benford's Law: 30% (wash trading is the biggest risk)
    - Order Book Accuracy: 25% (critical for execution)
    - Tick Completeness: 20% (data gaps corrupt backtests)
    - Buy/Sell Symmetry: 15% (market balance)
    - Normalization: 10% (timestamp quality)
    """
    scores = {}
    
    # Benford's Law (30%)
    if exchange_id in benford_results:
        benford = benford_results[exchange_id]
        if benford['result'] == 'PASS':
            scores['benford'] = 100
        elif benford['result'] == 'SUSPICIOUS':
            scores['benford'] = 50
        elif benford['result'] == 'FAIL':
            scores['benford'] = 0
        else:
            scores['benford'] = 50
    else:
        scores['benford'] = 50
    
    # Order Book Accuracy (25%)
    if exchange_id in orderbook_results:
        scores['orderbook'] = orderbook_results[exchange_id]['score']
    else:
        scores['orderbook'] = 0
    
    # Tick Completeness (20%)
    if exchange_id in tick_results:
        scores['tick'] = tick_results[exchange_id]['score']
    else:
        scores['tick'] = 0
    
    # Buy/Sell Symmetry (15%)
    if exchange_id in symmetry_results:
        symmetry = symmetry_results[exchange_id]
        if symmetry['result'] == 'PASS':
            scores['symmetry'] = 100
        elif symmetry['result'] == 'ACCEPTABLE':
            scores['symmetry'] = 70
        else:
            scores['symmetry'] = 30
    else:
        scores['symmetry'] = 50
    
    # Normalization (10%)
    if exchange_id in normalization_results:
        scores['normalization'] = normalization_results[exchange_id]['score']
    else:
        scores['normalization'] = 0
    
    # Weighted composite score
    trust_score = (
        scores['benford'] * 0.30 +
        scores['orderbook'] * 0.25 +
        scores['tick'] * 0.20 +
        scores['symmetry'] * 0.15 +
        scores['normalization'] * 0.10
    )
    
    # Grade
    if trust_score >= 90:
        grade = 'A+'
    elif trust_score >= 80:
        grade = 'A'
    elif trust_score >= 70:
        grade = 'B'
    elif trust_score >= 60:
        grade = 'C'
    elif trust_score >= 50:
        grade = 'D'
    else:
        grade = 'F'
    
    return {
        'trust_score': trust_score,
        'grade': grade,
        'component_scores': scores
    }

# Calculate trust scores for all exchanges
trust_scores = {}
for exchange_id in EXCHANGES_TO_TEST:
    trust_scores[exchange_id] = calculate_trust_score(exchange_id)

print("\n🏆 Composite Data Trust Scores:\n")
print(f"{'Exchange':<12} {'Score':>8} {'Grade':>6} {'Benford':>8} {'OB':>8} {'Tick':>8} {'Sym':>8} {'Norm':>8}")
print("=" * 80)
for exchange, data in sorted(trust_scores.items(), key=lambda x: x[1]['trust_score'], reverse=True):
    cs = data['component_scores']
    print(f"{exchange:<12} {data['trust_score']:>8.1f} {data['grade']:>6} {cs['benford']:>8.0f} {cs['orderbook']:>8.0f} {cs['tick']:>8.0f} {cs['symmetry']:>8.0f} {cs['normalization']:>8.0f}")

---

## Section 5: Visualizations

Interactive Plotly charts (dark mode) for data exploration.

In [ ]:
# 1. Trust Score Heatmap
def create_trust_score_heatmap():
    """Create heatmap of all quality metrics."""
    exchanges = list(trust_scores.keys())
    metrics = ['Benford', 'Order Book', 'Tick Complete', 'Buy/Sell Sym', 'Normalization']
    
    # Build matrix
    matrix = []
    for exchange in exchanges:
        cs = trust_scores[exchange]['component_scores']
        matrix.append([
            cs['benford'],
            cs['orderbook'],
            cs['tick'],
            cs['symmetry'],
            cs['normalization']
        ])
    
    fig = go.Figure(data=go.Heatmap(
        z=matrix,
        x=metrics,
        y=exchanges,
        colorscale='RdYlGn',
        text=matrix,
        texttemplate='%{text:.0f}',
        textfont={"size": 10},
        colorbar=dict(title="Score")
    ))
    
    fig.update_layout(
        title='Exchange Data Quality Heatmap (0-100 Scale)',
        template='plotly_dark',
        height=600,
        xaxis_title='Quality Metric',
        yaxis_title='Exchange'
    )
    
    return fig

heatmap_fig = create_trust_score_heatmap()
heatmap_fig.show()
heatmap_fig.write_html('/home/ubuntu/.openclaw/workspace/algohouse-data-quality-benchmark/heatmap.html')
print("\n✅ Heatmap saved to heatmap.html")

In [ ]:
# 2. Trust Score vs. Volume Scatter Plot
def create_trust_vs_volume_scatter():
    """Scatter plot: Trust Score vs. 24h Volume."""
    exchanges_list = []
    trust_list = []
    volume_list = []
    
    for exchange in EXCHANGES_TO_TEST:
        if exchange in trust_scores and exchange in exchange_data:
            exchanges_list.append(exchange.upper())
            trust_list.append(trust_scores[exchange]['trust_score'])
            volume_list.append(exchange_data[exchange].get('volume_24h', 0))
    
    fig = go.Figure(data=go.Scatter(
        x=volume_list,
        y=trust_list,
        mode='markers+text',
        text=exchanges_list,
        textposition='top center',
        marker=dict(
            size=12,
            color=trust_list,
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Trust Score")
        )
    ))
    
    fig.update_layout(
        title='Data Trust Score vs. 24h Volume',
        template='plotly_dark',
        height=600,
        xaxis_title='24h Volume (USDT)',
        yaxis_title='Data Trust Score (0-100)',
        xaxis_type='log'
    )
    
    return fig

scatter_fig = create_trust_vs_volume_scatter()
scatter_fig.show()
scatter_fig.write_html('/home/ubuntu/.openclaw/workspace/algohouse-data-quality-benchmark/scatter.html')
print("\n✅ Scatter plot saved to scatter.html")

In [ ]:
# 3. Trust Score Bar Chart (Ranked)
def create_trust_score_bar_chart():
    """Bar chart of trust scores ranked high to low."""
    sorted_exchanges = sorted(trust_scores.items(), key=lambda x: x[1]['trust_score'], reverse=True)
    
    exchanges = [e[0].upper() for e in sorted_exchanges]
    scores = [e[1]['trust_score'] for e in sorted_exchanges]
    grades = [e[1]['grade'] for e in sorted_exchanges]
    
    colors = ['#00cc66' if s >= 80 else '#ffcc00' if s >= 60 else '#ff6666' for s in scores]
    
    fig = go.Figure(data=go.Bar(
        x=exchanges,
        y=scores,
        text=[f"{s:.1f} ({g})" for s, g in zip(scores, grades)],
        textposition='outside',
        marker_color=colors
    ))
    
    fig.update_layout(
        title='Exchange Data Trust Scores (Ranked)',
        template='plotly_dark',
        height=600,
        xaxis_title='Exchange',
        yaxis_title='Trust Score (0-100)',
        yaxis_range=[0, 110]
    )
    
    return fig

bar_fig = create_trust_score_bar_chart()
bar_fig.show()
bar_fig.write_html('/home/ubuntu/.openclaw/workspace/algohouse-data-quality-benchmark/barchart.html')
print("\n✅ Bar chart saved to barchart.html")

---

## Summary Report

In [ ]:
# Generate summary report
print("\n" + "="*80)
print("📊 ALGOHOUSE DATA QUALITY BENCHMARK - SUMMARY REPORT")
print("="*80)

print(f"\n📅 Benchmark Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🎯 Symbol Tested: {SYMBOL}")
print(f"🔢 Exchanges Tested: {len(EXCHANGES_TO_TEST)}")
print(f"📈 Trades Analyzed: {sum(d.get('trade_count', 0) for d in exchange_data.values())}")

print("\n🏆 TOP 3 EXCHANGES BY TRUST SCORE:")
top_3 = sorted(trust_scores.items(), key=lambda x: x[1]['trust_score'], reverse=True)[:3]
for rank, (exchange, data) in enumerate(top_3, 1):
    print(f"  {rank}. {exchange.upper():12} - Score: {data['trust_score']:.1f}/100 (Grade: {data['grade']})")

print("\n⚠️  EXCHANGES FLAGGED FOR WASH TRADING (Benford's Law):")
flagged = [e for e, r in benford_results.items() if r['result'] in ['FAIL', 'SUSPICIOUS']]
if flagged:
    for exchange in flagged:
        result = benford_results[exchange]
        print(f"  - {exchange.upper():12} ({result['result']}, p-value: {result.get('p_value', 0):.4f})")
else:
    print("  ✅ No exchanges flagged")

print("\n📊 EXPORTS:")
print("  - heatmap.html (quality metrics heatmap)")
print("  - scatter.html (trust vs. volume)")
print("  - barchart.html (ranked trust scores)")

print("\n" + "="*80)
print("✅ Benchmark complete! Runtime: < 10 minutes")
print("="*80)